In [1]:
from distributed import Client
import os
#import logging
os.environ["MODIN_ENGINE"] = "dask"
client = Client(silence_logs=50)

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
import modin.pandas as pd
import re
import tensorflow as tf
import intel_extension_for_tensorflow as itex
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering, TFAutoModelForSequenceClassification, set_seed
from datasets import Dataset
from transformers import create_optimizer
import string
import os
import random

2023-10-20 03:52:06.471130: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-20 03:52:06.689962: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-20 03:52:07.642824: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-20 03:52:07.646213: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-20 03:52:15.330206: W tensorflow/compiler/tf2t

In [4]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XPU:0', device_type='XPU')]

In [5]:
SEED = 42
set_seed = SEED
tf.keras.utils.set_random_seed(42)
random.seed(SEED)

In [6]:
BASE_DIR = '/home/u132668/'

In [7]:
train = pd.read_csv(os.path.join(BASE_DIR, 'data/train.csv'))

In [8]:
train.drop(['span_start','span_end','span_text'], axis=1, inplace=True)

In [9]:
def preprocess_story(text):
    #text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub('--', ' ', text)
    text = re.sub('\s{2,}', ' ', text)
    text = text.strip()
    text = text.lower()
    return text

In [10]:
train['Story'] = train['Story'].apply(preprocess_story)
train['Question'] = train['Question'].apply(lambda x: x.lower().strip().strip(string.punctuation))
train['Answer'] = train['Answer'].apply(lambda x: x.lower().strip().strip(string.punctuation))

In [11]:
train_tf = train[(train['Answer'] == 'true') | (train['Answer'] == 'false')].copy()
train_yn = train[(train['Answer'] == 'yes') | (train['Answer'] == 'no')].copy()

In [12]:
train_qa = train.drop(train_tf.index)
train_qa = train_qa.drop(train_yn.index)
train_qa = train_qa.drop(train[train['Answer'] == 'unknown'].index)

In [13]:
train_qa['answer_start'] = train_qa.apply(lambda row: row['Story'].rfind(row['Answer']), axis=1)

In [14]:
train_story_contained_ans = train_qa[train_qa['answer_start']>=0]

# QA Training:

In [15]:
model_checkpoint = "bert-large-uncased-whole-word-masking-finetuned-squad"

In [16]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

2023-10-20 03:54:22.339356: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform XPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-20 03:54:22.339537: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:XPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: XPU, pci bus id: <undefined>)
All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

All the weights of TFBertForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


In [17]:
dataset = Dataset.from_pandas(train_story_contained_ans._to_pandas())

In [18]:
dataset = dataset.train_test_split(test_size=0.2, seed=SEED, shuffle=True)

In [19]:
pad_on_right = True
max_length = 512
doc_stride = 128

def prepare_train_features(examples):

    tokenized_examples = tokenizer(
        examples["Question" if pad_on_right else "Story"],
        examples["Story" if pad_on_right else "Question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )


    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")


    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):

        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        sequence_ids = tokenized_examples.sequence_ids(i)
        sample_index = sample_mapping[i]
        answers = examples["Answer"][sample_index]
        
        if examples["Answer"][sample_index] == -1:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            start_char = examples["answer_start"][sample_index]
            end_char = start_char + len(answers)
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                while (token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char):
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [20]:
tokenized_data = dataset.map(prepare_train_features, batched=True, remove_columns=dataset["train"].column_names)

Map:   0%|          | 0/35279 [00:00<?, ? examples/s]

Map:   0%|          | 0/8820 [00:00<?, ? examples/s]

In [21]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 35378
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 8843
    })
})

In [22]:
batch_size = 8
num_epochs = 2
total_train_steps = (len(tokenized_data["train"]) // batch_size) * num_epochs
optimizer, schedule = create_optimizer(
    init_lr=3e-5,
    num_warmup_steps=0,
    num_train_steps=total_train_steps,
    #weight_decay_rate = 0.01
)

In [23]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_data["train"],
    batch_size=batch_size
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_data["test"],
    batch_size=batch_size
)

In [24]:
model.summary()

Model: "tf_bert_for_question_answering"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  334092288 
                                                                 
 qa_outputs (Dense)          multiple                  2050      
                                                                 
Total params: 334,094,338
Trainable params: 334,094,338
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.layers[0].trainable = False

In [26]:
model.compile(optimizer=optimizer, metrics=["accuracy"])

In [27]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=num_epochs)

Epoch 1/2


2023-10-20 04:09:15.519449: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type XPU is enabled.


4422/4422 [==============================] - ETA: 0s - loss: 2.4656 - end_logits_accuracy: 0.4710 - start_logits_accuracy: 0.4189

2023-10-20 04:28:52.550322: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type XPU is enabled.


4422/4422 [==============================] - 1482s 325ms/step - loss: 2.4656 - end_logits_accuracy: 0.4710 - start_logits_accuracy: 0.4189 - val_loss: 2.2246 - val_end_logits_accuracy: 0.5103 - val_start_logits_accuracy: 0.4548
Epoch 2/2
4422/4422 [==============================] - 1400s 317ms/step - loss: 2.3381 - end_logits_accuracy: 0.4801 - start_logits_accuracy: 0.4324 - val_loss: 2.1970 - val_end_logits_accuracy: 0.5132 - val_start_logits_accuracy: 0.4554


In [28]:
model.save_pretrained(os.path.join(BASE_DIR,'saved_models/QA/v2'))
tokenizer.save_pretrained(os.path.join(BASE_DIR,'saved_models/QA/v2'))

('/home/u132668/saved_models/QA/v2/tokenizer_config.json',
 '/home/u132668/saved_models/QA/v2/special_tokens_map.json',
 '/home/u132668/saved_models/QA/v2/vocab.txt',
 '/home/u132668/saved_models/QA/v2/added_tokens.json',
 '/home/u132668/saved_models/QA/v2/tokenizer.json')

In [29]:
model.push_to_hub("WaRKiD/bert-large-uncased-whole-word-masking-finetuned-intel-oneapi-llm-dataset")
tokenizer.push_to_hub("WaRKiD/bert-large-uncased-whole-word-masking-finetuned-intel-oneapi-llm-dataset")

tf_model.h5:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/WaRKiD/bert-large-uncased-whole-word-masking-finetuned-intel-oneapi-llm-dataset/commit/711cfa75c2a0507226fde202e9b17fcb72c0a242', commit_message='Upload tokenizer', commit_description='', oid='711cfa75c2a0507226fde202e9b17fcb72c0a242', pr_url=None, pr_revision=None, pr_num=None)